<div class="frontmatter text-center">
<h1> Introduction to Data Science and Programming</h1>
<h2>Lecture 20: Skewed data</h2>
<h3>IT University of Copenhagen, Fall 2020</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Source
This notebook was adapted from:
* James Bagrow: http://bagrow.com/dsv/

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  
from random import *
np.random.seed(seed=10)

# From narrow to broad data

### Narrow data

In many physical systems a quantity is distributed following a normal distribution tightly around some central value. For example, the heights of humans.

You can see this distribution with a histogram:

In [ ]:
heights = 1.75 + np.random.randn(1000,1)*0.1

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(heights, 20, edgecolor="w")
axes.set_xlabel("Height [m]")
axes.set_ylabel("Number of subjects");

It's called the "normal distribution" because it's so common.

You don't see humans that are 20 meters tall, or 20 kilometers.

### Broad data

But many systems, like social systems, do not exhibit such a narrow spread of values!

USA income distribution:

<img src="files/us-income-distribution.gif" width=800>

If we compute a histogram going from 5k/year of income to 500M/year, we're going to need a huge number of bins to cover that range, and almost all of the data is going to be crammed into the first few bins (covering everyone earning less than, say, 1M/year).

***

**Broad data** covers a very large range of values. Much larger than you would see from a normal distribution.

It is **skewed** in one direction. You can see if this is happening by comparing the mean and the median:

In [ ]:
np.random.seed(seed=10)
incomes = 10000*(1+np.random.pareto(1.0, 1000)) # fake data that somewhat models real income distributions
incomes = incomes.astype(int)

print("   min = " + str(min(incomes)))
print("   max = " + str(max(incomes)))
print("  mean = " + str(np.mean(incomes)))
print("median = " + str(np.median(incomes)))

In [ ]:
incomes

If we try to make a histogram, we're **going to have a bad time**:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(incomes, 30, edgecolor="w")
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group");

We can use more bins and zoom in, but we're still not going to see much:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(incomes, 500, edgecolor="w")
axes.set_xlim([10000,200000])
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group");

We need a log transformation!

## Log transformation

The function $y= \log(x)$ grows very very slowly. 
We can make $x$ **HUGE** and the log will just **cram** it way down to a small number.

Let's see this:

In [ ]:
x = np.linspace(1,1000,400)
y = np.log10(x)

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.plot(x, y, '-', lw=3)
axes.grid(True,which="both",ls="-")
axes.set_xlabel("x")
axes.set_ylabel("y = log(x)")
axes.set_title("Linear plot");

### Log transforming the plot scale

We now plot **exactly** the same data, just on a semilogx scale:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.semilogx(x, y, '-', lw=3)
axes.grid(True,which="both",ls="-")
axes.set_xlabel("x")
axes.set_ylabel("y = log(x)")
axes.set_title("Nonlinear plot");

We can **change space** as we move along the x-axis, squashing larger and larger chunks into a given visual length in such as way that the slow-growing log function now **appears linear!**

### Log transforming the data itself

Instead of squeezing the plot scale, we could just squeeze the data, and use a linear plot scale. The visual outcome is the same:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.plot(np.log10(x), y, '-', lw=3 )
axes.grid(True,which="both",ls="-")
axes.set_xlabel("log(x)")
axes.set_ylabel("y = log(x)")
axes.set_title("Linear plot on log-transformed data");

The only difference is that `semilogx` is smart enough to nicely label the x-axis with $10^0$ instead of $0$, $10^1$ instead of $1$, $10^2$ instead of $2$, etc., and to allow displaying a logarithmic grid.

### Log transforming the income data

So what? Why is this helpful?

The income data is too broad to see what is happening. The log is a **squashing** function.  

We can log-transform the broad data to squash it down into a range where we can see what's going on!

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 3))

# Left subplot: Original income data
axes = ax[0]
axes.hist(incomes, 30, edgecolor="w")
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group")

# Right subplot: log transformed income data
axes = ax[1]
axes.hist(np.log10(incomes), 30, edgecolor="w")
axes.set_xlabel("log(Income)")
axes.set_ylabel("Number of people in log(income) group");

Nice! We see much more now. And we can even spot the millionaires at 6 ($10^6$).

### Plotting a histogram in log scale

What happens if we bin the original income data and then plot that on a log-axis?

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(incomes, 30, edgecolor="w")
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group");

axes.set_xscale('log') 
axes.grid(True,which="both",ls="-") # Shown for teaching

**Whoa** That ain't right. 

All we've done is taken the original bins and stretched them logarithmically. That one huge bin still contains the vast majority of the data.

***
To make a nice plot, what we should do is choose **non-uniform** bins. If you choose bins that are spaced wider and wider apart, the log will squash them down until (if chosen well) they are all the same width on the logarithmic scale:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(incomes,  bins=np.logspace(4, 7, 30), edgecolor="w")
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group");
axes.set_xscale('log') 
axes.grid(True,which="both",ls="-") # Shown for teaching

### loglog scale

Sometimes both your x- and your y-values are broadly distribued. We can also log-transform y.

If we plot a function $y=f(x)$ on a plot where both the x- and the y-scales are logarithmic and we see a **straight line** what kind of function is $f$?


Ans: power function.


$$y = a x^b$$

To see this take the log of both sides and replace $\log(y) \to y'$ and $\log(x) \to x'$.

$$
\log(y) = \log(a x^b) = \log(a) + \log(x^b) = a' + b \log(x)
$$

so

$$y' = a' + b x'$$

which is a straight line when we plot $x'$ vs. $y'$. The slope of the straight line is $b$, the exponent of $x$.

*** 

Let's see this in action:

In [ ]:
minx = min(incomes)
maxx = max(incomes)
num_bins = 50
logbins = np.logspace(np.log10(minx), np.log10(maxx), num_bins)

fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

axes.hist(incomes,  bins=logbins, edgecolor="w")
axes.set_xlabel("Income")
axes.set_ylabel("Number of people in income group");
axes.set_xscale('log') 
axes.set_yscale('log') 
axes.grid(True,which="both",ls="-") # Shown for teaching

Nice. 

***

But notice how we don't really need bars, and how they are infinitely tall due to $log(0) \to -\infty$? Therefore it is more customary to draw a plot using lines or points. Also it is nicer to draw a density function using `density=True`:

In [ ]:
fig = plt.figure(figsize=(4, 3))
axes = fig.add_axes([0, 0, 1, 1])

h, binedges = np.histogram(incomes,  bins=logbins, density=True) # We use np.histogram to make the binning calculations. Plotting is done with axes.plot below.
binmidpoints = (binedges[1:] + binedges[:-1]) / 2
axes.plot(binmidpoints, h, "o")
axes.set_xlabel("Income x")
axes.set_ylabel("f(x)");
axes.set_xscale('log') 
axes.set_yscale('log') 
axes.grid(True,which="both",ls="-") # Shown for teaching

We can see well the power law relation here, but there are also fluctuations. Going next to the complementary cumulative density function solves this issue.

*Back to presentation*

# From probability density function (PDF) to complementary cumulative density function (CCDF)

In [ ]:
def plot_pdfcdfccdf(x, setylog=False, setxlog=False, n_bins=100):
    """Plots the PDF, CDF, and CCDF for a given data set x, using hist.""" 

    fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(16, 3))
    xlim = [np.min(x),np.max(x)]
    if setxlog:
        xlogstring = "log "
    else:
        xlogstring = ""

    # Plot the density curve
    ax = axes[0]
    ax.hist(x, n_bins, density=True, cumulative=False, log=setylog)
    ax.set_ylabel('$f(x)$')
    ax.set_xlabel('Data '+ xlogstring +'$x$')
    ax.set_title('PDF')

    # Plot the cumulative density curve
    ax = axes[1]
    ax.hist(x, n_bins, density=True, cumulative=True, log=setylog)
    ax.set_ylabel('$P_{\leq}(x)$')
    ax.set_xlabel('Data '+ xlogstring +'$x$')
    ax.set_title('CDF')

    # Plot the complementary cumulative density curve
    ax = axes[2]
    ax.hist(x, n_bins, density=True, cumulative=-1, log=setylog)
    ax.set_ylabel('$P_{>}(x)$')
    ax.set_xlabel('Data '+ xlogstring +'$x$')
    ax.set_title('CCDF')

    for ax in axes:
        ax.set_xlim(xlim)

### Uniform distribution

#### Theoretical

In [ ]:
plot_pdfcdfccdf(range(100))

#### Sampled

In [ ]:
x = np.random.uniform(0, 100, size=10000)
plot_pdfcdfccdf(x)

Nice: the CDF and CCDF get rid of all the fluctuations.

### Normal distribution

In [ ]:
x = np.random.normal(175, 5, size=10000)
plot_pdfcdfccdf(x)

### Exponential distribution

#### Linear scale

In [ ]:
x = np.random.exponential(100, size=10000)
plot_pdfcdfccdf(x)

#### Semilogy scale (better)

In [ ]:
plot_pdfcdfccdf(x, True)

Nice: We can see how the exponential becomes a straight line in the semilogy PDF and CCDF. The CCDF has the added value of being monotonically non-increasing: it is getting rid of fluctuations.

Also note how on a semilogy scale we really need to go from CDF to CCDF so that we can investigate the tail.

### Power law distribution

#### Linear scale (bad)

In [ ]:
plot_pdfcdfccdf(incomes)

#### Semilogy scale (better)

In [ ]:
plot_pdfcdfccdf(incomes, True)

#### Loglog (best)

In [ ]:
plot_pdfcdfccdf(np.log10(incomes), True, True)

Nice: We can see how the power law becomes a straight line in the loglog PDF and CCDF. The CCDF has the added value of being monotonically non-increasing: it is getting rid of fluctuations.

Also note how on a loglog scale we really need to go from CDF to CCDF so that we can investigate the tail.

***
Look how different this is to the normal distribution in loglog scale:

In [ ]:
x = np.random.normal(175, 5, size=10000)
plot_pdfcdfccdf(np.log10(x), True, True)

The range of log x values is **miniscule**: 2.19 to 2.29! The tail drops off *extremely* fast.

## Beyond binning

To make the CCDF plot adequate, we should give up binning as it can distort conclusions: You can "lie" by choosing a certain binning. Also, a CCDF is usually drawn using a line and/or point markers. The `hist` method is not really adequate for all this - we could write our own script to create adequate CCDF plots. 

In any case, this notebook should give you the important idea behind the log transformation and the CCDF.

*Back to presentation*